In [17]:
#use anytree instead, it has nice prints / plots
#https://pypi.org/project/anytree/
class Tree():
    def __init__(self, data, parent):
        self.data = data
        self.parent = parent
        self.children = []
    def __eq__(self, other):
        if isinstance(other, Tree):
            return self.data == other.data
        else:
            return False
    def __repr__(self):
        return "Tree("+str(self.data)+","+str(self.children)+")"
    def __str__(self):
        return self.__repr__()
    def update_parent(self, new):
        self.parent = new
    def add_child(self, c):
        self.children.append(c)
    def rm_child(self, c):
        self.children.remove(c)
    

In [18]:
#sl subtemplate of bl (matching to the same or a subset of patterns)
def sub(sl, bl):
    if sl==bl: return True
    if len(bl)==0: return False
    if len(sl)==0:
        b, *bs = bl
        if b == "*":
            return sub(sl, bs)
        return False
    s, *ss = sl
    b, *bs = bl
    if b=="*":
        if s == "*":
            return sub(ss, bs) or sub(ss, bl)
        else:
            return sub(ss, bs) or sub(ss, bl) or sub(sl, bs)
    elif b=="_":
        if s=="*": return False
        else: return sub(ss, bs)
    else:
        if s == "*" or s == "_": return False
        elif s == b: return sub(ss, bs)
        else: return False

In [19]:
#test cases
print(sub(["I", "_"],["I", "RW"]))
print(sub(["I", "RW"], ["I", "_"]))
print(sub(["I", "_"],["I", "RW", "M"]))
print(sub(["I", "RW", "M"], ["I", "_"]))
print(sub(["*", "_"],["I", "RW", "M"]))
print(sub(["I", "RW", "M"],["*", "_"]))
print(sub(["*"],["I", "RW", "*"]))
print(sub(["I", "RW", "*"], ["*"]))

False
True
False
False
False
True
False
True


In [20]:
#get the parent candidate for a template, i.e. a node with a template that matches a superset but no child matches a superset
def get_parent_can(tree, template):
    if tree.data == template:
        return tree
    if sub(template, tree.data):
        if len(tree.children)==0:
            return tree
        tmp = [get_parent_can(c, template) for c in tree.children]
        tmp = [x for x in tmp if x!=False]
        if len(tmp)==0:
            return tree
        elif len(tmp)==1:
            return tmp[0]
        else:
            raise ValueError('more than one child can be parent of '+str(template)+"   "+str(tree.children))
    else:
        return False

In [21]:
#insert templeate into the tree:
#find parent candidate
#if it has the same template noting changes
#otherwise check wether some children need to be reorganized and insert the new node properly
def update_tree(tree, template):
    new_parent = get_parent_can(tree, template)
    if new_parent.data == template:
        return tree
    tmp = [sub(c.data, template) for c in new_parent.children]
    if any(tmp):
        rm = []
        new_node = Tree(template, new_parent)
        for i in range(len(tmp)):
            if tmp[i]:
                moved = new_parent.children[i]
                rm.append(moved)
                new_node.add_child(moved)
                moved.update_parent(new_node)
        for i in rm:
            new_parent.rm_child(i)
        new_parent.add_child(new_node)
    else:
        new_parent.add_child(Tree(template, new_parent))

In [22]:
#each template is a child, only the root template "*" is more general
templates = {("I","M","C"),("I","RW","M","C"),("I","M","M","F","C")}
tree = Tree(["*"], None)
tree

Tree(['*'],[])

In [23]:
for templ in templates:
    update_tree(tree, templ)

In [24]:
tree

Tree(['*'],[Tree(('I', 'M', 'C'),[]), Tree(('I', 'RW', 'M', 'C'),[]), Tree(('I', 'M', 'M', 'F', 'C'),[])])

In [25]:
#need to insert new templates as inner nodes into the tree, as they are more general than the current leaves
templates2 = {("I","*","C"),("I","RW","*","C"),("I","RW","M", "*","C")}
for templ in templates2:
    update_tree(tree, templ)

In [26]:
tree

Tree(['*'],[Tree(('I', '*', 'C'),[Tree(('I', 'M', 'C'),[]), Tree(('I', 'M', 'M', 'F', 'C'),[]), Tree(('I', 'RW', '*', 'C'),[Tree(('I', 'RW', 'M', '*', 'C'),[Tree(('I', 'RW', 'M', 'C'),[])])])])])

In [27]:
from anytree import Node, RenderTree

In [28]:
def t2anytree(tree):
    if len(tree.children)>0:
        ch_nodes = [t2anytree(x) for x in tree.children]
        return Node(str(tree.data), children=ch_nodes)
    else:
        return Node(str(tree.data))

In [29]:
atree = t2anytree(tree)

In [30]:
print(RenderTree(atree))

Node("/['*']")
└── Node("/['*']/('I', '*', 'C')")
    ├── Node("/['*']/('I', '*', 'C')/('I', 'M', 'C')")
    ├── Node("/['*']/('I', '*', 'C')/('I', 'M', 'M', 'F', 'C')")
    └── Node("/['*']/('I', '*', 'C')/('I', 'RW', '*', 'C')")
        └── Node("/['*']/('I', '*', 'C')/('I', 'RW', '*', 'C')/('I', 'RW', 'M', '*', 'C')")
            └── Node("/['*']/('I', '*', 'C')/('I', 'RW', '*', 'C')/('I', 'RW', 'M', '*', 'C')/('I', 'RW', 'M', 'C')")


In [31]:
for pre, _, node in RenderTree(atree):
    print("%s%s" % (pre, node.name))

['*']
└── ('I', '*', 'C')
    ├── ('I', 'M', 'C')
    ├── ('I', 'M', 'M', 'F', 'C')
    └── ('I', 'RW', '*', 'C')
        └── ('I', 'RW', 'M', '*', 'C')
            └── ('I', 'RW', 'M', 'C')


In [32]:
from anytree.exporter import DotExporter
# graphviz needs to be installed for the next line!, #N: via apt, not pip
DotExporter(atree).to_picture("tree.png")